In [ ]:
import os
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.optim import Adam, SGD
import matplotlib.pyplot as plt

In [ ]:
from model import Net
from tdnn import TDNN
from dataloader import DataLoader

In [ ]:
device = torch.device("cuda:1")
torch.cuda.is_available()

In [ ]:
def eval_accuracy(model, data_val, num_batches=None):
    model.eval()
    criterion = nn.CrossEntropyLoss().to(device)
    y_top1 = np.zeros(0)
    y_top3 = np.zeros((0, 3))
    losses = []
    top1_accs = []
    top3_accs = []
    for inputs, targets, lengths in data_val():
        with torch.no_grad():
            output = model(inputs, lengths)
            loss = criterion(output, targets)
            losses.append(loss.item())
            y_top1 = np.argmax(output.cpu().detach().numpy(), axis=1)
            y_top3 = np.argpartition(output.cpu().detach().numpy(), -3, axis=1)[:, -3:]
            top1_accs.append(np.mean(targets.cpu().detach().numpy() == y_top1))
        if num_batches:
            num_batches -= 1
            if num_batches == 0:
                break
    loss = np.mean(losses)
    top1_acc = np.mean(top1_accs)
    model.train()
    return loss, top1_acc

In [ ]:
dummy = Net(5994)
print("Number of parameters:", sum(p.numel() for p in dummy.parameters() if p.requires_grad))

In [ ]:
data_path = 'mfcc'
file_name = 'mfcc'
y = np.load(data_path + '/speakers.npy')

In [ ]:
val_size = 0.1

data_size = y.shape[0]
rand_idx = np.random.choice(data_size, size=data_size, replace=False)
border = int((1-val_size)*data_size)
train_idx = rand_idx[:border]
val_idx = rand_idx[border:]

In [ ]:
batch_size = 16
num_epochs = 100

data_train = DataLoader(data_path, file_name, y, train_idx, batch_size, device=device)
data_val = DataLoader(data_path, file_name, y, val_idx, batch_size, device=device)

n_speakers = len(np.unique(y))
model = Net(n_speakers).to(device)
optimizer = Adam(model.parameters(), lr=1e-4, weight_decay=1e-6)
criterion = nn.CrossEntropyLoss().to(device)

print(f'\repoch: 0/{num_epochs}, evaluating...', end='')
train_loss, train_acc = eval_accuracy(model, data_train, num_batches=500)
val_loss, val_acc = eval_accuracy(model, data_val, num_batches=500)
print(f'\repoch: 0/{num_epochs}, '
      f'train loss: {"%.5f"%train_loss}, train acc: {"%.5f"%train_acc}',
      f'val loss: {"%.5f"%val_loss}, val acc: {"%.5f"%val_acc}')

for epoch in range(num_epochs):
    losses = []
    i = 0
    best_loss = 5
    for inputs, targets, lengths in data_train():
        i += 1
        outputs = model(inputs, lengths)
        optimizer.zero_grad()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        print(f'\repoch: {epoch+1}/{num_epochs}, batch: {i}/{train_idx.shape[0]//batch_size+1}, size: {inputs.shape[1]}', end='')
    if (epoch + 1) % 1 == 0:
        print(f'\repoch: {epoch+1}/{num_epochs}, train loss: {"%.5f"%np.mean(losses)}, evaluating...', end='')
        train_loss, train_acc = eval_accuracy(model, data_train, num_batches=5000)
        val_loss, val_acc = eval_accuracy(model, data_val, num_batches=5000)
        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model, f'models/full_model{epoch+1}.pkl')
        print(f'\repoch: {epoch+1}/{num_epochs}, '
              f'train loss: {"%.5f"%train_loss}, train acc: {"%.5f"%train_acc}',
              f'val loss: {"%.5f"%val_loss}, val acc: {"%.5f"%val_acc}')